In [17]:
# mkdir if not exist 'proposals_out_2023_07_21'
import os
if not os.path.exists('proposals_out_2023_07_21'):
    os.makedirs('proposals_out_2023_07_21')

In [18]:
# read in ../../DATA/deployments/snapshot_deployments.csv
import pandas as pd
df = pd.read_csv('../../DATA/deployments/snapshot_deployments.csv')
space_ids = df.platform_id.unique()
len(space_ids)

19615

In [19]:
df.head()

,platform,platform_id,name,additional,website,votes_count,proposals_count
0,snapshot,sybilizer.eth,DAO,NaN,https://snapshot.org/#/sybilizer.eth,0,0
1,snapshot,sybuild.eth,vote,NaN,https://snapshot.org/#/sybuild.eth,1,1
2,snapshot,sydtek.eth,SydTek,NaN,https://snapshot.org/#/sydtek.eth,0,0
3,snapshot,sylvial.eth,dsfsd,NaN,https://snapshot.org/#/sylvial.eth,1,1
4,snapshot,sysin.eth,Testing,NaN,https://snapshot.org/#/sysin.eth,1,1


In [20]:
df.proposals_count.describe()

count    20782.000000
mean         6.523771
std         35.320325
min          0.000000
25%          1.000000
50%          1.000000
75%          3.000000
max       2816.000000
Name: proposals_count, dtype: float64

In [21]:
# remove the space_ids that have no proposals
df = df[df.proposals_count > 0]
platforms_and_proposal_count = df[['platform_id', 'proposals_count']].sort_values(by='proposals_count', ascending=False)
# make these into a list of tuples
platforms_and_proposal_count = list(platforms_and_proposal_count.itertuples(index=False, name=None))
platforms_and_proposal_count[:5]

[('cakevote.eth', 2816),
 ('snapshot.dcl.eth', 1984),
 ('index-coop.eth', 919),
 ('gm365.eth', 731),
 ('frami.eth', 637)]

In [22]:
# make groups of rows where the proposals_count in the group sums to less than 5000
groups_of_space_ids_that_sum_to_less_than_5000 = []
while len(platforms_and_proposal_count) > 0:
    group = []
    total = 0
    while total < 5000:
        if platforms_and_proposal_count[0][1] + total > 5000:
            break
        row = platforms_and_proposal_count.pop(0)
        group.append(row)
        total += row[1]
        if len(platforms_and_proposal_count) == 0:
            break
    groups_of_space_ids_that_sum_to_less_than_5000.append(group)

groups_of_space_ids_that_sum_to_less_than_5000

[[('cakevote.eth', 2816), ('snapshot.dcl.eth', 1984)],
 [('index-coop.eth', 919),
  ('gm365.eth', 731),
  ('frami.eth', 637),
  ('apollodao.eth', 634),
  ('huwacoin.eth', 628),
  ('maturka.eth', 626),
  ('maturka.eth', 626)],
 [('toniya.eth', 528),
  ('balancer.eth', 520),
  ('bancornetwork.eth', 500),
  ('sdbal.eth', 451),
  ('ppyaa.eth', 428),
  ('lemu.dcl.eth', 423),
  ('eth6888.eth', 420),
  ('bentfinance.eth', 420),
  ('sharkdao.eth', 413),
  ('aavegotchi.eth', 410),
  ('14cat.eth', 376)],
 [('aave.eth', 368),
  ('rosette093510.eth', 367),
  ('haitangkeji.eth', 365),
  ('2021eth.eth', 364),
  ('2021eth.eth', 364),
  ('fabien.eth', 356),
  ('goodmorningnews.eth', 352),
  ('dao-dvision.eth', 342),
  ('abcdcba.eth', 341),
  ('zys888.eth', 335),
  ('frax.eth', 333),
  ('boboat321.eth', 330),
  ('✓✓✓✓✓.eth', 329),
  ('bnbdddd.eth', 329)],
 [('jbdao.eth', 329),
  ('wdswap.eth', 326),
  ('loveeachother.eth', 324),
  ('•8099.eth', 323),
  ('iwan.eth', 322),
  ('skgame.eth', 321),
  ('aura

In [23]:
len(groups_of_space_ids_that_sum_to_less_than_5000)

28

In [24]:
# sum the proposals_count in each group
sums = [sum([x[1] for x in group]) for group in groups_of_space_ids_that_sum_to_less_than_5000]
sums

[4800,
 4801,
 4889,
 4875,
 4903,
 4958,
 4904,
 4912,
 4983,
 4994,
 4990,
 4964,
 4960,
 4964,
 4997,
 4990,
 4991,
 4991,
 4998,
 4999,
 4993,
 4998,
 4995,
 5000,
 5000,
 5000,
 5000,
 1728]

In [25]:
# make into a list of strings for the graphql query
groups_of_space_ids_that_sum_to_less_than_5000_strings = []
for group in groups_of_space_ids_that_sum_to_less_than_5000:
    group_of_space_ids = [str(x[0]) for x in group]
    query_string = ", ".join([f'"{sid}"' for sid in group_of_space_ids])
    groups_of_space_ids_that_sum_to_less_than_5000_strings.append(query_string)

groups_of_space_ids_that_sum_to_less_than_5000_strings[0]

'"cakevote.eth", "snapshot.dcl.eth"'

In [26]:
def make_query(space_ids: str, skip: int) -> str:
    first = 1000
    base = """
        query Proposals {
         proposals(where: {space_in: [%s], state: "closed"}, first: %s, skip: %s, orderDirection: desc, orderBy: "created") {
            id
            space {
              id
            }
            author
            created
            votes
          }
        }
    """
    return base % (space_ids, first, skip)


In [27]:
import requests
import backoff
URL = "https://hub.snapshot.org/graphql"


@backoff.on_exception(backoff.expo,
                      requests.exceptions.RequestException)
def query_snapshot(query):
    r = requests.post(URL, json={'query': query})
    return r.json()

In [28]:
import json
for count, space_id_group in enumerate(groups_of_space_ids_that_sum_to_less_than_5000_strings):
    # make pagination logic
    skip = 0
    has_more = True
    while has_more:
        filename = f'proposals_out_2023_07_21/proposals_{count}_{skip}.json'
        next_proposals_filename = f'proposals_out_2023_07_21/proposals_{count}_{skip + 1000}.json'
        if os.path.exists(next_proposals_filename):
            print(f'next file exists: {next_proposals_filename}')
            skip += 1000
            continue
        next_group_filename = f'proposals_out_2023_07_21/proposals_{count + 1}_0.json'
        if os.path.exists(next_group_filename):
            print(f'next group file exists: {next_group_filename}')
            has_more = False
            continue
        query = make_query(space_id_group, skip)
        result = query_snapshot(query)
        proposals = result['data']['proposals']
        print('got', len(proposals), 'proposals for', count, skip)
        # save proposals to file in the output dir
        with open(filename, 'w') as f:
            json.dump(proposals, f)
        if len(proposals) == 1000:
            skip += 1000
        else:
            has_more = False


got 1000 proposals for 0 0
got 1000 proposals for 0 1000
got 1000 proposals for 0 2000
got 1000 proposals for 0 3000
got 706 proposals for 0 4000
got 1000 proposals for 1 0
got 1000 proposals for 1 1000
got 1000 proposals for 1 2000
got 1000 proposals for 1 3000
got 132 proposals for 1 4000
got 1000 proposals for 2 0
got 1000 proposals for 2 1000
got 1000 proposals for 2 2000
got 1000 proposals for 2 3000
got 851 proposals for 2 4000
got 1000 proposals for 3 0
got 1000 proposals for 3 1000
got 1000 proposals for 3 2000
got 1000 proposals for 3 3000
got 477 proposals for 3 4000
got 1000 proposals for 4 0
got 1000 proposals for 4 1000
got 1000 proposals for 4 2000
got 1000 proposals for 4 3000
got 859 proposals for 4 4000
got 1000 proposals for 5 0
got 1000 proposals for 5 1000
got 1000 proposals for 5 2000
got 1000 proposals for 5 3000
got 898 proposals for 5 4000
got 1000 proposals for 6 0
got 1000 proposals for 6 1000
got 1000 proposals for 6 2000
got 1000 proposals for 6 3000
got 877